<a href="https://colab.research.google.com/github/singhtejn/Stock_AI_ML/blob/main/Install_Ta_lib_on_Google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install ta-lib python wrapper on Google colab

Ta-lib python wrapper : https://github.com/mrjbq7/ta-lib

Dependencies :
> To use TA-Lib for python, you need to have the **TA-Lib already installed**. ...




In [ ]:
# download TA-Lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!ls

!tar xvzf ta-lib-0.4.0-src.tar.gz

!ls

import os

os.chdir('ta-lib') # Can't use !cd in co-lab

!./configure --prefix=/usr

!make

!make install
# wait ~ 30s

os.chdir('../')

!ls

!pip install TA-Lib

import talib

In [ ]:
# prompt: install matplotlib finance

!pip install mplfinance


No. of days hammer pattern is observed

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import datetime

import matplotlib.dates as mdates
import mplfinance as mpf # import mplfinance instead of matplotlib.finance

from matplotlib.dates import MONDAY, DateFormatter, DayLocator, WeekdayLocator
from datetime import timedelta

def plot_pattern(quotes, title, pattern_name, index):
    # quotes : pandas dataframe
    # index : annotation index in
    # sample : plot_pattern(AP_df, title='Test', pattern_name='Hanging Man', index=100)

    mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
    alldays = DayLocator()              # minor ticks on the days
    weekFormatter = DateFormatter('%b %d')  # e.g., Jan 12
    dayFormatter = DateFormatter('%d')      # e.g., 12

    l = quotes.shape[0]
    d1 = index + 16 if index + 8 < l else l - 1  # 7 days after
    d2 = index - 15 if index - 7 >= 0 else 0     # 7 days before

    # select desired range of dates
    quotes2 = quotes.iloc[d2:d1]

    fig, ax = plt.subplots()
    fig.subplots_adjust(bottom=0.2)
    ax.xaxis.set_major_locator(mondays)
    ax.xaxis.set_minor_locator(alldays)
    ax.xaxis.set_major_formatter(weekFormatter)
    # ax.xaxis.set_minor_formatter(dayFormatter)

    mpf.plot(quotes2, type='candle', ax=ax, style='charles',
             show_nontrading=True, datetime_format='%Y-%m-%d',
             xrotation=45, volume=False)

    ax.xaxis_date()
    ax.autoscale_view()
    fig.set_size_inches(10, 5)
    plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    ax.set_title(title)
    ymin, ymax = ax.get_ylim()
    selected_date = quotes.iloc[[index]].index.to_pydatetime()

    ax.annotate(pattern_name,
                xy=(selected_date, quotes.loc[selected_date].high),
                xytext=(selected_date, quotes.loc[selected_date].high + (ymax - ymin) / 4),
                arrowprops=dict(facecolor='black', width=0.5, shrink=0.25))
    plt.show()

# Example usage with your dataframe df
open = df['open'].values
high = df['high'].values
low = df['low'].values
close = df['close'].values
result = talib.CDLHAMMER(open, high, low, close)

result

hammer_days = np.where(result != 0)[0]
for i in hammer_days:
    plot_pattern(df, 'Hammer day ' + str(i), 'Hammer', i)


No. of days hammer pattern is observed

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import talib
from matplotlib.dates import MONDAY, DateFormatter, DayLocator, WeekdayLocator

# Fetching AAPL stock data (you can replace this with your data source)
# Here, we use yfinance for demonstration
import yfinance as yf
aapl = yf.download('AAPL', start='2023-01-01', end='2024-07-31')

# Preprocess the data
aapl.reset_index(inplace=True)
aapl['Date'] = pd.to_datetime(aapl['Date'])
aapl.set_index('Date', inplace=True)

# Identify hammer patterns
open_prices = aapl['Open'].values
high_prices = aapl['High'].values
low_prices = aapl['Low'].values
close_prices = aapl['Close'].values
result = talib.CDLHAMMER(open_prices, high_prices, low_prices, close_prices)

# Extract indices of hammer pattern days
hammer_days = np.where(result != 0)[0]

def plot_pattern(quotes, title, pattern_name, index):
    mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
    alldays = DayLocator()              # minor ticks on the days
    weekFormatter = DateFormatter('%b %d')  # e.g., Jan 12

    l = quotes.shape[0]
    d1 = index + 16 if index + 8 < l else l - 1  # 7 days after
    d2 = index - 15 if index - 7 >= 0 else 0     # 7 days before

    # select desired range of dates
    quotes2 = quotes.iloc[d2:d1]

    fig, ax = plt.subplots()
    fig.subplots_adjust(bottom=0.2)
    ax.xaxis.set_major_locator(mondays)
    ax.xaxis.set_minor_locator(alldays)
    ax.xaxis.set_major_formatter(weekFormatter)

    mpf.plot(quotes2, type='candle', ax=ax, style='charles',
             show_nontrading=True, datetime_format='%Y-%m-%d',
             xrotation=45, volume=False)

    ax.xaxis_date()
    ax.autoscale_view()
    fig.set_size_inches(10, 5)
    plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
    ax.set_title(title)
    ymin, ymax = ax.get_ylim()
    selected_date = quotes.iloc[[index]].index.to_pydatetime()[0]

    ax.annotate(pattern_name,
                xy=(selected_date, quotes.loc[selected_date]['High']),
                xytext=(selected_date, quotes.loc[selected_date]['High'] + (ymax - ymin) / 4),
                arrowprops=dict(facecolor='black', width=0.5, shrink=0.25))
    plt.show()

# Plot hammer patterns
for i in hammer_days:
    plot_pattern(aapl, 'Hammer day ' + str(i), 'Hammer', i)


No. of times certain patterns are observed

In [ ]:
import pandas as pd
import yfinance as yf
import talib
import numpy as np

# Fetch daily NVDA stock data for the last 2 years
nvda = yf.download('NVDA', start='2022-01-01', end='2024-01-01', interval='1d')

# Ensure the datetime columns are timezone-naive
nvda.reset_index(inplace=True)
nvda['Date'] = pd.to_datetime(nvda['Date']).dt.tz_localize(None)
nvda.set_index('Date', inplace=True)

# Define the bullish patterns available in TA-Lib
bullish_patterns = {
    'CDL2CROWS': 'Two Crows',
    'CDL3BLACKCROWS': 'Three Black Crows',
    'CDL3INSIDE': 'Three Inside Up/Down',
    'CDL3LINESTRIKE': 'Three-Line Strike',
    'CDL3OUTSIDE': 'Three Outside Up/Down',
    'CDL3STARSINSOUTH': 'Three Stars In The South',
    'CDL3WHITESOLDIERS': 'Three Advancing White Soldiers',
    'CDLABANDONEDBABY': 'Abandoned Baby',
    'CDLADVANCEBLOCK': 'Advance Block',
    'CDLBELTHOLD': 'Belt-hold',
    'CDLBREAKAWAY': 'Breakaway',
    'CDLCLOSINGMARUBOZU': 'Closing Marubozu',
    'CDLCONCEALBABYSWALL': 'Concealing Baby Swallow',
    'CDLCOUNTERATTACK': 'Counterattack',
    'CDLDARKCLOUDCOVER': 'Dark Cloud Cover',
    'CDLDOJI': 'Doji',
    'CDLDOJISTAR': 'Doji Star',
    'CDLDRAGONFLYDOJI': 'Dragonfly Doji',
    'CDLENGULFING': 'Engulfing Pattern',
    'CDLEVENINGDOJISTAR': 'Evening Doji Star',
    'CDLEVENINGSTAR': 'Evening Star',
    'CDLGAPSIDESIDEWHITE': 'Up/Down-gap side-by-side white lines',
    'CDLGRAVESTONEDOJI': 'Gravestone Doji',
    'CDLHAMMER': 'Hammer',
    'CDLHANGINGMAN': 'Hanging Man',
    'CDLHARAMI': 'Harami Pattern',
    'CDLHARAMICROSS': 'Harami Cross Pattern',
    'CDLHIGHWAVE': 'High-Wave Candle',
    'CDLHIKKAKE': 'Hikkake Pattern',
    'CDLHIKKAKEMOD': 'Modified Hikkake Pattern',
    'CDLHOMINGPIGEON': 'Homing Pigeon',
    'CDLIDENTICAL3CROWS': 'Identical Three Crows',
    'CDLINNECK': 'In-Neck Pattern',
    'CDLINVERTEDHAMMER': 'Inverted Hammer',
    'CDLKICKING': 'Kicking',
    'CDLKICKINGBYLENGTH': 'Kicking - bull/bear determined by the longer marubozu',
    'CDLLADDERBOTTOM': 'Ladder Bottom',
    'CDLLONGLEGGEDDOJI': 'Long Legged Doji',
    'CDLLONGLINE': 'Long Line Candle',
    'CDLMARUBOZU': 'Marubozu',
    'CDLMATCHINGLOW': 'Matching Low',
    'CDLMATHOLD': 'Mat Hold',
    'CDLMORNINGDOJISTAR': 'Morning Doji Star',
    'CDLMORNINGSTAR': 'Morning Star',
    'CDLONNECK': 'On-Neck Pattern',
    'CDLPIERCING': 'Piercing Pattern',
    'CDLRICKSHAWMAN': 'Rickshaw Man',
    'CDLRISEFALL3METHODS': 'Rising/Falling Three Methods',
    'CDLSEPARATINGLINES': 'Separating Lines',
    'CDLSHOOTINGSTAR': 'Shooting Star',
    'CDLSHORTLINE': 'Short Line Candle',
    'CDLSPINNINGTOP': 'Spinning Top',
    'CDLSTALLEDPATTERN': 'Stalled Pattern',
    'CDLSTICKSANDWICH': 'Stick Sandwich',
    'CDLTAKURI': 'Takuri (Dragonfly Doji with very long lower shadow)',
    'CDLTASUKIGAP': 'Tasuki Gap',
    'CDLTHRUSTING': 'Thrusting Pattern',
    'CDLTRISTAR': 'Tristar Pattern',
    'CDLUNIQUE3RIVER': 'Unique 3 River',
    'CDLUPSIDEGAP2CROWS': 'Upside Gap Two Crows',
    'CDLXSIDEGAP3METHODS': 'Upside/Downside Gap Three Methods'
}

# Initialize a dictionary to count the occurrences of each bullish pattern
pattern_counts = {pattern: 0 for pattern in bullish_patterns.keys()}

# Calculate the occurrences of each bullish pattern
for pattern in bullish_patterns.keys():
    pattern_function = getattr(talib, pattern)
    result = pattern_function(nvda['Open'], nvda['High'], nvda['Low'], nvda['Close'])
    pattern_counts[pattern] = np.count_nonzero(result)

# Convert the pattern counts to a DataFrame for easy display
pattern_counts_df = pd.DataFrame.from_dict(pattern_counts, orient='index', columns=['Count'])
pattern_counts_df['Pattern Name'] = pattern_counts_df.index.map(bullish_patterns)
pattern_counts_df.reset_index(drop=True, inplace=True)
pattern_counts_df = pattern_counts_df.sort_values(by='Count', ascending=False)

print(pattern_counts_df)

output_file = "/content/nvda_bullish_patterns.csv"
pattern_counts_df.to_csv(output_file, index=False)

output_file


Accuracy of trading patterns only

In [ ]:
import pandas as pd
import yfinance as yf
import talib
import numpy as np

# Fetch daily NVDA stock data for the last 2 years
nvda = yf.download('SAIL.NS', start='2022-08-31', end='2024-01-01', interval='1h')


# Ensure the datetime columns are timezone-naive
nvda.reset_index(inplace=True)
nvda['Datetime'] = pd.to_datetime(nvda['Datetime']).dt.tz_localize(None)
nvda.set_index('Datetime', inplace=True)

# Define the bullish patterns available in TA-Lib
bullish_patterns = {
    'CDL2CROWS': 'Two Crows',
    'CDL3BLACKCROWS': 'Three Black Crows',
    'CDL3INSIDE': 'Three Inside Up/Down',
    'CDL3LINESTRIKE': 'Three-Line Strike',
    'CDL3OUTSIDE': 'Three Outside Up/Down',
    'CDL3STARSINSOUTH': 'Three Stars In The South',
    'CDL3WHITESOLDIERS': 'Three Advancing White Soldiers',
    'CDLABANDONEDBABY': 'Abandoned Baby',
    'CDLADVANCEBLOCK': 'Advance Block',
    'CDLBELTHOLD': 'Belt-hold',
    'CDLBREAKAWAY': 'Breakaway',
    'CDLCLOSINGMARUBOZU': 'Closing Marubozu',
    'CDLCONCEALBABYSWALL': 'Concealing Baby Swallow',
    'CDLCOUNTERATTACK': 'Counterattack',
    'CDLDARKCLOUDCOVER': 'Dark Cloud Cover',
    'CDLDOJI': 'Doji',
    'CDLDOJISTAR': 'Doji Star',
    'CDLDRAGONFLYDOJI': 'Dragonfly Doji',
    'CDLENGULFING': 'Engulfing Pattern',
    'CDLEVENINGDOJISTAR': 'Evening Doji Star',
    'CDLEVENINGSTAR': 'Evening Star',
    'CDLGAPSIDESIDEWHITE': 'Up/Down-gap side-by-side white lines',
    'CDLGRAVESTONEDOJI': 'Gravestone Doji',
    'CDLHAMMER': 'Hammer',
    'CDLHANGINGMAN': 'Hanging Man',
    'CDLHARAMI': 'Harami Pattern',
    'CDLHARAMICROSS': 'Harami Cross Pattern',
    'CDLHIGHWAVE': 'High-Wave Candle',
    'CDLHIKKAKE': 'Hikkake Pattern',
    'CDLHIKKAKEMOD': 'Modified Hikkake Pattern',
    'CDLHOMINGPIGEON': 'Homing Pigeon',
    'CDLIDENTICAL3CROWS': 'Identical Three Crows',
    'CDLINNECK': 'In-Neck Pattern',
    'CDLINVERTEDHAMMER': 'Inverted Hammer',
    'CDLKICKING': 'Kicking',
    'CDLKICKINGBYLENGTH': 'Kicking - bull/bear determined by the longer marubozu',
    'CDLLADDERBOTTOM': 'Ladder Bottom',
    'CDLLONGLEGGEDDOJI': 'Long Legged Doji',
    'CDLLONGLINE': 'Long Line Candle',
    'CDLMARUBOZU': 'Marubozu',
    'CDLMATCHINGLOW': 'Matching Low',
    'CDLMATHOLD': 'Mat Hold',
    'CDLMORNINGDOJISTAR': 'Morning Doji Star',
    'CDLMORNINGSTAR': 'Morning Star',
    'CDLONNECK': 'On-Neck Pattern',
    'CDLPIERCING': 'Piercing Pattern',
    'CDLRICKSHAWMAN': 'Rickshaw Man',
    'CDLRISEFALL3METHODS': 'Rising/Falling Three Methods',
    'CDLSEPARATINGLINES': 'Separating Lines',
    'CDLSHOOTINGSTAR': 'Shooting Star',
    'CDLSHORTLINE': 'Short Line Candle',
    'CDLSPINNINGTOP': 'Spinning Top',
    'CDLSTALLEDPATTERN': 'Stalled Pattern',
    'CDLSTICKSANDWICH': 'Stick Sandwich',
    'CDLTAKURI': 'Takuri (Dragonfly Doji with very long lower shadow)',
    'CDLTASUKIGAP': 'Tasuki Gap',
    'CDLTHRUSTING': 'Thrusting Pattern',
    'CDLTRISTAR': 'Tristar Pattern',
    'CDLUNIQUE3RIVER': 'Unique 3 River',
    'CDLUPSIDEGAP2CROWS': 'Upside Gap Two Crows',
    'CDLXSIDEGAP3METHODS': 'Upside/Downside Gap Three Methods'
}

# Initialize a dictionary to count the occurrences and success of each bullish pattern
pattern_success = {pattern: {'count': 0, 'success': 0} for pattern in bullish_patterns.keys()}

# Check if the stock price increases by 2% within 5 days after each pattern
for pattern in bullish_patterns.keys():
    pattern_function = getattr(talib, pattern)
    result = pattern_function(nvda['Open'], nvda['High'], nvda['Low'], nvda['Close'])
    pattern_indices = np.where(result != 0)[0]

    for index in pattern_indices:
        if index + 2 < len(nvda):
            future_prices = nvda['Close'][index+1:index+6]
            if any(future_prices >= nvda['Close'][index] * 1.01):
                pattern_success[pattern]['success'] += 1
        pattern_success[pattern]['count'] += 1

# Calculate success rates
for pattern in pattern_success.keys():
    if pattern_success[pattern]['count'] > 0:
        pattern_success[pattern]['success_rate'] = pattern_success[pattern]['success'] / pattern_success[pattern]['count']
    else:
        pattern_success[pattern]['success_rate'] = 0

# Convert the pattern success dictionary to a DataFrame for easy display
pattern_success_df = pd.DataFrame.from_dict(pattern_success, orient='index')
pattern_success_df['Pattern Name'] = pattern_success_df.index.map(bullish_patterns)
pattern_success_df.reset_index(drop=True, inplace=True)
pattern_success_df = pattern_success_df.sort_values(by='success_rate', ascending=False)

# Save the results to a CSV file
output_file = "nvda_bullish_pattern_success.csv"
pattern_success_df.to_csv(output_file, index=False)

pattern_success_df
